# Hyperparameters

Most algoriths have **hyperparameters**. For some optimization methods the parameters are already defined and can directly be optimized. For instance, for Differential Evolution (DE) the parameters can be found by:

In [1]:
import json
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.core.parameters import get_params, flatten, set_params, hierarchical

algorithm = DE()
flatten(get_params(algorithm))

{'mating.jitter': <pymoo.core.variable.Choice at 0x7fddb9d20f10>,
 'mating.CR': <pymoo.core.variable.Real at 0x7fddb9d20eb0>,
 'mating.crossover': <pymoo.core.variable.Choice at 0x7fddb9b32ac0>,
 'mating.F': <pymoo.core.variable.Real at 0x7fddb9d20e20>,
 'mating.n_diffs': <pymoo.core.variable.Choice at 0x7fddb9d20dc0>,
 'mating.selection': <pymoo.core.variable.Choice at 0x7fddb9d20d60>}

If not provided directly, when initializing a `HyperparameterProblem` these variables are directly used for optimization.

Secondly, one needs to define what exactly should be optimized. For instance, for a single run on a problem (with a fixed random seed) using the well-known parameter optimization toolkit [Optuna](https://optuna.org), the implementation may look as follows:

In [2]:
from pymoo.algorithms.hyperparameters import SingleObjectiveSingleRun, HyperparameterProblem
from pymoo.algorithms.soo.nonconvex.g3pcx import G3PCX
from pymoo.algorithms.soo.nonconvex.optuna import Optuna
from pymoo.core.parameters import set_params, hierarchical
from pymoo.optimize import minimize
from pymoo.problems.single import Sphere

algorithm = G3PCX()

problem = Sphere(n_var=10)
n_evals = 500

performance = SingleObjectiveSingleRun(problem, termination=("n_evals", n_evals), seed=1)

res = minimize(HyperparameterProblem(algorithm, performance),
               Optuna(),
               termination=('n_evals', 50),
               seed=1,
               verbose=False)

hyperparams = res.X
print(hyperparams)
set_params(algorithm, hierarchical(hyperparams))

res = minimize(Sphere(), algorithm, termination=("n_evals", n_evals), seed=1)
print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

{'mutation.eta': 24.01087920319256, 'mutation.prob': 0.31947746041945063, 'crossover.zeta': 0.19712354833387533, 'crossover.eta': 0.06510354041781266, 'family_size': 9, 'n_parents': 10, 'n_offsprings': 2, 'pop_size': 122}
Best solution found: 
X = [0.49931263 0.49971117 0.49995666 0.50012005 0.49995485 0.49954006
 0.50013753 0.49992644 0.50004025 0.50003723]
F = [8.13108862e-07]


Of course, you can also directly use the `MixedVariableGA` available in our framework:

In [3]:
from pymoo.algorithms.hyperparameters import SingleObjectiveSingleRun, HyperparameterProblem
from pymoo.algorithms.soo.nonconvex.g3pcx import G3PCX
from pymoo.algorithms.soo.nonconvex.optuna import Optuna
from pymoo.core.mixed import MixedVariableGA
from pymoo.core.parameters import set_params, hierarchical
from pymoo.optimize import minimize
from pymoo.problems.single import Sphere


algorithm = G3PCX()

problem = Sphere(n_var=10)
n_evals = 500

performance = SingleObjectiveSingleRun(problem, termination=("n_evals", n_evals), seed=1)

res = minimize(HyperparameterProblem(algorithm, performance),
               MixedVariableGA(pop_size=5),
               termination=('n_evals', 50),
               seed=1,
               verbose=False)

hyperparams = res.X
print(hyperparams)
set_params(algorithm, hierarchical(hyperparams))

res = minimize(Sphere(), algorithm, termination=("n_evals", n_evals), seed=1)
print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

{'mutation.eta': 10.202478525408438, 'mutation.prob': 0.18217181670454258, 'crossover.zeta': 0.2672212328003948, 'crossover.eta': 0.038505365248462986, 'family_size': 10, 'n_parents': 4, 'n_offsprings': 5, 'pop_size': 169}
Best solution found: 
X = [0.49931151 0.49958918 0.49947128 0.50076286 0.50029319 0.50035433
 0.4995032  0.49959396 0.49910168 0.50016305]
F = [2.96104366e-06]


Now, optimizing the parameters for a **single random seed** is often not desirable. And this is precisely what makes hyper-parameter optimization computationally expensive. So instead of using just a single random seed, we can use the `MultiRun` performance assessment to average over multiple runs as follows:

In [4]:
from pymoo.algorithms.hyperparameters import HyperparameterProblem, MultiRun, stats_single_objective_mean
from pymoo.algorithms.soo.nonconvex.g3pcx import G3PCX
from pymoo.core.mixed import MixedVariableGA
from pymoo.core.parameters import set_params, hierarchical
from pymoo.optimize import minimize
from pymoo.problems.single import Sphere


algorithm = G3PCX()

problem = Sphere(n_var=10)
n_evals = 500
seeds = [5, 50, 500]

performance = MultiRun(problem, seeds=seeds, func_stats=stats_single_objective_mean, termination=("n_evals", n_evals))

res = minimize(HyperparameterProblem(algorithm, performance),
               MixedVariableGA(pop_size=5),
               termination=('n_evals', 50),
               seed=1,
               verbose=True)

hyperparams = res.X
print(hyperparams)
set_params(algorithm, hierarchical(hyperparams))

res = minimize(Sphere(), algorithm, termination=("n_evals", n_evals), seed=5)
print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))


n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |        5 |  0.0011469191 |  0.0001158058
     2 |       10 |  0.0003004380 |  1.324378E-06
     3 |       15 |  0.0001083746 |  1.324378E-06
     4 |       20 |  0.0000604004 |  1.324378E-06
     5 |       25 |  6.940028E-06 |  1.324378E-06
     6 |       30 |  2.363478E-06 |  1.324378E-06
     7 |       35 |  2.184765E-06 |  1.324378E-06
     8 |       40 |  1.801311E-06 |  1.290965E-06
     9 |       45 |  1.457222E-06 |  7.690855E-07
    10 |       50 |  1.457222E-06 |  7.690855E-07
{'mutation.eta': 3.003840525176007, 'mutation.prob': 0.2613802441259011, 'crossover.zeta': 0.2092756091839343, 'crossover.eta': 0.13100894947710487, 'family_size': 9, 'n_parents': 7, 'n_offsprings': 4, 'pop_size': 63}
Best solution found: 
X = [0.50058063 0.50049996 0.49944047 0.49958107 0.50008284 0.50007215
 0.49972702 0.50003434 0.49996608 0.50038491]
F = [1.31273882e-06]


Another way of performance measure is the number of evaluations until a specific goal has been reached. For single-objective optimization, such a goal is most likely until a minimum function value has been found. Thus, for the termination, we use `MinimumFunctionValueTermination` with a value of `1e-5`. We run the method for each random seed until this value has been reached or at most `500` function evaluations have taken place. The performance is then measured by the average number of function evaluations (`func_stats=stats_avg_nevals`) to reach the goal.

In [5]:
from pymoo.algorithms.hyperparameters import HyperparameterProblem, MultiRun, stats_avg_nevals
from pymoo.algorithms.soo.nonconvex.g3pcx import G3PCX
from pymoo.core.mixed import MixedVariableGA
from pymoo.core.parameters import set_params, hierarchical
from pymoo.core.termination import TerminateIfAny
from pymoo.optimize import minimize
from pymoo.problems.single import Sphere
from pymoo.termination.fmin import MinimumFunctionValueTermination
from pymoo.termination.max_eval import MaximumFunctionCallTermination

algorithm = G3PCX()

problem = Sphere(n_var=10)

termination = TerminateIfAny(MinimumFunctionValueTermination(1e-5), MaximumFunctionCallTermination(500))

performance = MultiRun(problem, seeds=[5, 50, 500], func_stats=stats_avg_nevals, termination=termination)

res = minimize(HyperparameterProblem(algorithm, performance),
               MixedVariableGA(pop_size=5),
               ('n_evals', 50),
               seed=1,
               verbose=True)

hyperparams = res.X
print(hyperparams)
set_params(algorithm, hierarchical(hyperparams))

res = minimize(Sphere(), algorithm, termination=("n_evals", res.f), seed=5)
print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))


n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |        5 |  5.298000E+02 |  5.030000E+02
     2 |       10 |  4.895333E+02 |  4.276667E+02
     3 |       15 |  4.763333E+02 |  4.276667E+02
     4 |       20 |  4.571333E+02 |  4.276667E+02
     5 |       25 |  4.411333E+02 |  4.276667E+02
     6 |       30 |  4.382000E+02 |  4.276667E+02
     7 |       35 |  4.360667E+02 |  4.276667E+02
     8 |       40 |  4.200667E+02 |  3.850000E+02
     9 |       45 |  4.068667E+02 |  3.850000E+02
    10 |       50 |  4.068667E+02 |  3.850000E+02
{'mutation.eta': 22.546590161392896, 'mutation.prob': 0.13789985939149485, 'crossover.zeta': 0.20871365511506024, 'crossover.eta': 0.12039717249146664, 'family_size': 10, 'n_parents': 7, 'n_offsprings': 4, 'pop_size': 65}
Best solution found: 
X = [0.5005912  0.499759   0.50062838 0.49857344 0.49949559 0.50094711
 0.49905868 0.49954207 0.49892447 0.50003376]
F = [6.24263744e-06]
